In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso,Ridge,ElasticNet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler,minmax_scale,Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import PowerTransformer
from sklearn import tree

In [3]:
dir='/content/drive/MyDrive/Crop/Data/training_data.csv'
df=pd.read_csv(dir,index_col=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9436 entries, 0 to 9435
Columns: 769 entries, Year to AccumulatedTT_pMatHar
dtypes: float64(762), int64(5), object(2)
memory usage: 55.4+ MB


In [34]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

In [6]:
# Encode field
field_encoder = preprocessing.LabelEncoder()
field_encoder.fit(df['Field_Location'])
df['Field_Location'] = field_encoder.transform(df['Field_Location'])
# Encode hybrid
hybrid_encoder = preprocessing.LabelEncoder()
hybrid_encoder.fit(df['Hybrid'])
df['Hybrid'] = hybrid_encoder.transform(df['Hybrid'])

In [33]:
data = df[df['Year'] == 2021]
X=data.drop(columns=['Yield_Mg_ha'], axis=1)
Y=data['Yield_Mg_ha']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)
print(x_train.shape)
print(y_train.shape)

(1776, 768)
(1776,)


In [35]:
# Train model with default alpha=1
lasso = Lasso().fit(x_train, y_train)

# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)

grid = GridSearchCV(estimator=lasso, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Y_pred_test = grid.predict(x_test)
r2 = r2_score(y_test, Y_pred_test) 
print('R2 Score:',r2)
rmse = mean_squared_error(y_true=y_test, y_pred=Y_pred_test, squared=False)
print('RMSE: ',rmse)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.476e+03, tolerance: 2.478e+00
  model = cd_fast.enet_coordinate_descent(


Best Score:  0.3270946048614305
Best Params:  {'alpha': 0.1}
R2 Score: 0.3691785665693055
RMSE:  2.777644682871809


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.594e+03, tolerance: 2.478e+00
  model = cd_fast.enet_coordinate_descent(


In [36]:
# Train model with default alpha=1
ridge = Ridge().fit(x_train, y_train)

# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)

grid = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)
Y_pred_test = grid.predict(x_test)
r2 = r2_score(y_test, Y_pred_test) 
print('R2 Score:',r2)
rmse = mean_squared_error(y_true=y_test, y_pred=Y_pred_test, squared=False)
print('RMSE: ',rmse)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best Score:  0.32539406144772876
Best Params:  {'alpha': 1000}
R2 Score: 0.37006363701797584
RMSE:  2.775695419565391


In [37]:
# Train model with default alpha=1 and l1_ratio=0.5
elastic_net_model = ElasticNet(alpha=1, l1_ratio=0.5).fit(x_train, y_train)
# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)

grid = GridSearchCV(estimator=elastic_net_model, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)
Y_pred_test = grid.predict(x_test)
r2 = r2_score(y_test, Y_pred_test) 
print('R2 Score:',r2)
rmse = mean_squared_error(y_true=y_test, y_pred=Y_pred_test, squared=False)
print('RMSE: ',rmse)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.271e+03, tolerance: 2.478e+00
  model = cd_fast.enet_coordinate_descent(


Fitting 5 folds for each of 77 candidates, totalling 385 fits
Best Score:  0.32744597546797927
Best Params:  {'alpha': 10, 'l1_ratio': 0}
R2 Score: 0.3693410936903182
RMSE:  2.777286838598334


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.138e+03, tolerance: 2.478e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [38]:
# Train model with default alpha=1
model = RandomForestRegressor().fit(x_train, y_train)

# find optimal alpha with grid search
max_features = [2,4,6,8]
param_grid = dict(max_features=max_features)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)
Y_pred_test = grid.predict(x_test)
r2 = r2_score(y_test, Y_pred_test) 
print('R2 Score:',r2)
rmse = mean_squared_error(y_true=y_test, y_pred=Y_pred_test, squared=False)
print('RMSE: ',rmse)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score:  0.5260278488885801
Best Params:  {'max_features': 2}
R2 Score: 0.5084962025180451
RMSE:  2.4518105430857764


In [39]:
from keras.layers import Bidirectional,LSTM
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Conv1D,concatenate,Rescaling

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from keras.utils.vis_utils import plot_model

import tensorflow as tf
import keras
from sklearn.metrics import mean_squared_error, r2_score
from keras.callbacks import EarlyStopping

In [44]:
print(x_train.shape[1])

768


In [40]:
batch_size = 128 #512
epochs = 10 #10
learning_rate = 1e-3 #1e-3

In [45]:
def create_model():
    model = keras.Sequential()
    model.add(keras.Input(shape=(X.shape[0],X.shape[1]),dtype="float32"))
    model.add(Bidirectional(LSTM(30)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Dense(1))

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=5,
    decay_rate=0.1)
    # optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    optimizer=tf.keras.optimizers.Adam(learning_rate)

    model.compile(loss = 'mean_squared_error',
                  optimizer = optimizer)
    return model

mymodel = create_model()
mymodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 60)               191760    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               7808      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 207,889
Trainable params: 207,889
Non-trainable params: 0
________________________________________________

In [46]:
history = mymodel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,validation_split=0.2)

Epoch 1/10


ValueError: ignored